In [39]:
%matplotlib inline


# Parrallel Boilerplate code taken from Mike's awesome class!!!
from IPython import parallel
from scipy import stats, integrate, spatial
import pandas as pd
# rc = parallel.Client()
# lv = rc.load_balanced_view()
# dview = rc[:]
# engine_id = xrange(len(lv))
# dview.scatter("engine_id", engine_id)
# print len(lv)

In [40]:
# with dview.sync_imports(): 
#     import numpy as np
#     import pandas as pd
#     from scipy import stats, integrate, spatial
#     import matplotlib.pyplot as plt
#     import seaborn as sns

In [41]:
df = pd.read_csv('train.csv')
dview.push({'df': df})

In [7]:
# dview.pull('df')

In [42]:
zeros = df[df.label == 0].drop('label', 1).reset_index().drop("index",1)
ones = df[df.label == 1].drop('label', 1).reset_index().drop("index",1)
twos = df[df.label == 2].drop('label', 1).reset_index().drop("index",1)
threes = df[df.label == 3].drop('label', 1).reset_index().drop("index",1)
fours = df[df.label == 4].drop('label', 1).reset_index().drop("index",1)
fives = df[df.label == 5].drop('label', 1).reset_index().drop("index",1)
sixs = df[df.label == 6].drop('label', 1).reset_index().drop("index",1)
sevens = df[df.label == 7].drop('label', 1).reset_index().drop("index",1)
eights = df[df.label == 8].drop('label', 1).reset_index().drop("index",1)
nines = df[df.label == 9].drop('label', 1).reset_index().drop("index",1)

dview.push({"zeros": zeros.values, "ones": ones.values, "twos": twos.values, "threes": threes.values, "fours": fours.values, "fives": fives.values, "sixs": sixs.values, "sevens": sevens.values, "eights": eights.values, "nines": nines.values})

In [43]:
# digit_list = [zeros, ones, twos, threes, fours, fives, sixs, sevens, eights, nines]

In [44]:
# for digit in digit_list:
#     sns.heatmap(data=pd.DataFrame(digit.head(1).values.reshape((28,28))))
#     plt.show()

In [45]:
zero = zeros.loc[0].reshape((28,28))
one = ones.loc[0].reshape((28,28))
two = twos.loc[0].reshape((28,28))
three = threes.loc[0].reshape((28,28))
four = fours.loc[0].reshape((28,28))
five = fives.loc[0].reshape((28,28))
six = sixs.loc[0].reshape((28,28))
seven = sevens.loc[0].reshape((28,28))
eight = eights.loc[0].reshape((28,28))
nine = nines.loc[0].reshape((28,28))

samples = [zero, one, two, three, four, five, six, seven, eight, nine]

In [46]:
# df.sort('label').label.value_counts().plot(kind='bar')

In [47]:
# def getNearest(theValue, anArrayOfValues): # Takes two NP arrays
#     minimum = None
#     for aValue in anArrayOfValues:
#         aValue = aValue.reshape((28,28))

#         distance = np.linalg.norm(theValue-aValue)
#         if distance == 0: # Skip self
#             continue

#         if minimum is None:
#             minimum = distance
#         else:
#             if minimum > distance:
#                 minimum = distance

#     return minimum

# getNearest(samples[0], digit_list[0].values)

# for sample in range(0, 10):
#     for digit in range(0, 10):
#         print "Compared " + str(sample) + " to " + str(digit)
#         print "Min: " + str(getNearest(samples[sample], digit_list[digit].values))

In [48]:
def getDistances(anArrayOfValues, otherArrayOfValues):
    import numpy as np
    distances = []
    for value in anArrayOfValues:
        for otherValue in otherArrayOfValues:
            distances.append(np.linalg.norm(value-otherValue))
    return distances

In [49]:
%%px
# Adjusting the index of i relative to which block is being computed
# Cross checked that result is (n(n-1))/2
def getDistances(anArrayOfValues, otherArrayOfValues, indexAdjust):
    import numpy as np
    distances = []
    for i in xrange(len(anArrayOfValues)):
        for j in xrange(i+indexAdjust+1,len(otherArrayOfValues)):
            distances.append(np.linalg.norm(anArrayOfValues[i]-otherArrayOfValues[j]))
    return distances

# Cross checked that result is n^2
def getDistancesNoDups(anArrayOfValues, otherArrayOfValues, indexAdjust):
    import numpy as np
    distances = []
    for i in xrange(len(anArrayOfValues)):
        for j in xrange(i+indexAdjust,len(otherArrayOfValues)):
            distances.append(np.linalg.norm(anArrayOfValues[i]-otherArrayOfValues[j]))
    return distances

In [50]:
# dview.scatter("scat_zeros", zeros.values)
# dview.scatter("scat_ones", ones.values)

In [51]:
# %%px 
# zeros_distances = getDistances(scat_zeros, zeros, engine_id[0]*len(scat_zeros))
# imposter_distances = getDistancesNoDups(scat_ones, zeros, engine_id[0]*len(scat_ones))
# ones_distances = getDistances(scat_ones, ones, engine_id[0]*len(scat_ones))

#### For all distance computations
Without dupliactes 1 loops, best of 3: 1min 45s per loop 4Cores

#### For zero distances only
Without dupliactes 1 loops, best of 1: 25.9 s per loop 4Cores

Without duplicated 1 loops, best of 3: 28.7 s per loop 4Cores

With duplicated 1 loops, best of 3: 36.6 s per loop 4Cores

#### For ones only
Without duplicates 1 loops, best of 3: 34.5 s per loop 4Cores

In [52]:
# len(dview.gather("zeros_distances").get())

In [53]:
# legit_ones = dview.gather("ones_distances").get()
# legit_zeros = dview.gather("zeros_distances").get()
# imposter_distances = dview.gather("imposter_distances").get()

#### Gathering the distances
1 loops, best of 3: 1min 17s per loop

In [54]:
# len(imposter_distances)
# len(legit_distances)

In [55]:
# plt.hist(legit_ones, bins=100, alpha=.5, label="genuine")
# plt.hist(imposter_distances, bins=100, alpha=.5, label="imposter")
# plt.show()

In [56]:
# plt.hist(legit_zeros, bins=100, alpha=.5, label="genuine")
# plt.hist(imposter_distances, bins=100, alpha=.5, label="imposter")
# plt.show()

In [57]:
# sortedLegitZeros = sorted(legit_zeros)
# sortedLegitOnes = sorted(legit_ones)
# sortedImposter = sorted(imposter_distances)

In [58]:
# def sortedROCPrep(legits, imposters, step): #sorted arrays
#     import math
#     Th = 0 #Threshold
#     jInd = 0
#     iInd = 0
#     # Initial state of FTW
#     Tp = 0.0
#     Fp = 0.0
#     Tn = len(imposters) - Fp
#     Fn = len(legits) - Tp
    
#     TPRs = []
#     TNRs = []
#     FPRs = []
    
#     while Th < max(imposters[-1], legits[-1]) :
#         for i in xrange(iInd,len(legits)):
#             if legits[i] <= Th:
#                 Tp += 1;
#                 Fn = len(legits) - Tp
#                 iInd += 1
#             else: 
#                 break
#         for j in xrange(jInd, len(imposters)):
#             if imposters[j] < Th:
#                 Fp += 1;
#                 Tn = len(imposters) - Fp
#                 jInd += 1
#             else:
#                 break
#         TPRs.append(float(Tp)/(Tp + Fn))
#         TNRs.append(float(Tn)/(Fp + Tn))
#         TNRs.append(float(FP)/(Fp + Tn))
#         Th += step
#     return ({"TPRs":TPRs,"TNRs": TNRs, "FPRs": FPRs})

In [59]:
# dataROCOnes = sortedROCPrep(sortedLegitOnes, sortedImposter, 100)
# dataROCZeros = sortedROCPrep(sortedLegitZeros, sortedImposter, 100)

In [60]:
# plt.figure()
# plt.plot(dataROCOnes[0], dataROCOnes[1])
# plt.ylabel("True Negative Rate")
# plt.xlabel("True Positive Rate")
# plt.show()

In [61]:
# plt.figure()
# plt.plot(dataROCZeros[0], dataROCZeros[1])
# plt.ylabel("True Negative Rate")
# plt.xlabel("True Positive Rate")
# plt.show()

In [90]:
def findDistances(df, df1, k): #Takes a DF with Label
    import scipy
    distances = scipy.spatial.distance.cdist(df, df)   
    return distances





In [2]:
def findNears(distances): #Matrix of distances
    mins = distances.argsort(axis=1)
    return mins

In [3]:
labels = df.label.values
distances = findDistances(df.drop("label", 1).values, df.drop("label", 1).values,4)
# mins = findNears(distances)
# accessed = [labels[x] for x in mins[:,0:5]]


NameError: name 'df' is not defined

In [4]:
distances.shape

NameError: name 'distances' is not defined

In [1]:
mins = findNears(distances)
accessed = [labels[x] for x in mins[:,0:5]]

NameError: name 'findNears' is not defined

In [95]:
accessed

[array([1, 1, 1, 1, 1]),
 array([0, 0, 0, 0, 0]),
 array([1, 1, 1, 1, 1]),
 array([4, 4, 4, 4, 4]),
 array([0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0]),
 array([7, 7, 7, 7, 7]),
 array([3, 5, 3, 3, 3]),
 array([5, 5, 5, 5, 5]),
 array([3, 3, 3, 3, 3]),
 array([8, 8, 8, 8, 8]),
 array([9, 9, 9, 9, 9]),
 array([1, 1, 1, 1, 1]),
 array([3, 3, 3, 3, 3]),
 array([3, 3, 3, 3, 3]),
 array([1, 1, 1, 1, 1]),
 array([2, 2, 2, 2, 2]),
 array([0, 0, 0, 0, 0]),
 array([7, 7, 7, 7, 7]),
 array([5, 5, 5, 5, 5]),
 array([8, 8, 8, 8, 8]),
 array([6, 6, 6, 6, 6]),
 array([2, 2, 2, 2, 2]),
 array([0, 0, 0, 0, 0]),
 array([2, 2, 2, 2, 2]),
 array([3, 3, 3, 3, 3]),
 array([6, 6, 6, 6, 6]),
 array([9, 9, 9, 9, 9]),
 array([9, 9, 9, 9, 9]),
 array([7, 7, 7, 7, 7]),
 array([8, 8, 8, 8, 8]),
 array([9, 9, 9, 9, 9]),
 array([4, 4, 4, 4, 4]),
 array([9, 9, 9, 9, 9]),
 array([2, 2, 2, 7, 7]),
 array([1, 1, 1, 1, 1]),
 array([3, 3, 3, 3, 3]),
 array([1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1]),
 array([4, 4, 4, 4, 4]),


In [195]:
l =["a","b","c","d","e"]
def randomSelector(the_list, sampleSize):
    the_list = the_list
    import random
    indexes = range(len(l))
    selected_idxs = random.sample(indexes, sampleSize)

    sample = []
    result = [l.pop(x) for x in sorted(selected_idxs, reverse=True)]
    
    return (the_list, result)

randomSelector(l, 2)

(['b', 'c', 'd'], ['e', 'a'])

In [199]:
import numpy as np
l =np.array(["a","b"],["c","d"],["e","f"])
l[[1,4,3]]

ValueError: only 2 non-keyword arguments accepted

In [207]:
from sklearn import cross_validation
kf = sk.cross_validation.KFold(len(accessed), n_folds=10)

In [210]:
kf.train_index

AttributeError: 'KFold' object has no attribute 'train_index'

In [214]:
for train_index, test_index in kf:
    print("TRAIN:", train_index, "TEST:", test_index)

('TRAIN:', array([ 4200,  4201,  4202, ..., 41997, 41998, 41999]), 'TEST:', array([   0,    1,    2, ..., 4197, 4198, 4199]))
('TRAIN:', array([    0,     1,     2, ..., 41997, 41998, 41999]), 'TEST:', array([4200, 4201, 4202, ..., 8397, 8398, 8399]))
('TRAIN:', array([    0,     1,     2, ..., 41997, 41998, 41999]), 'TEST:', array([ 8400,  8401,  8402, ..., 12597, 12598, 12599]))
('TRAIN:', array([    0,     1,     2, ..., 41997, 41998, 41999]), 'TEST:', array([12600, 12601, 12602, ..., 16797, 16798, 16799]))
('TRAIN:', array([    0,     1,     2, ..., 41997, 41998, 41999]), 'TEST:', array([16800, 16801, 16802, ..., 20997, 20998, 20999]))
('TRAIN:', array([    0,     1,     2, ..., 41997, 41998, 41999]), 'TEST:', array([21000, 21001, 21002, ..., 25197, 25198, 25199]))
('TRAIN:', array([    0,     1,     2, ..., 41997, 41998, 41999]), 'TEST:', array([25200, 25201, 25202, ..., 29397, 29398, 29399]))
('TRAIN:', array([    0,     1,     2, ..., 41997, 41998, 41999]), 'TEST:', array([29400

In [223]:
from scipy import stats
l = [1,1,2,3]
mode = stats.mode(l)

In [225]:
print mode

ModeResult(mode=array([1]), count=array([2]))
